### This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
<!--     for filename in filenames:
        print(os.path.join(dirname, filename)) -->

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing libraries   
import pandas as pd
import numpy as np
import cv2 
import os
import glob

In [ ]:
#importing Images
images_path="../input/flickr30k/Images/flickr30k_images/"
images=glob.glob(images_path+"*.jpg")
len(images)
images[:5]
import matplotlib.pyplot as plt
for i in range(5):
    plt.figure()
    img=cv2.imread(images[i])
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)

In [ ]:
#captions
caption_path = '../input/flickr30k/captions.txt'
captions = open(caption_path, 'rb').read().decode('utf-8').split('\n')
len(captions)

In [ ]:
captions[13]

In [ ]:
print(captions[:5])
print(len(captions[:5]))

In [ ]:
# seperate Image & Caption
print(f'{captions[:5][1]}')
img,caption = captions[:5][1].split(",")
print(img)

In [ ]:
pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)

In [ ]:
# seperate Image & Caption
print(f'{captions[13]}')
img,caption = captions[13].split(",")
print(img)

pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)

In [ ]:
# seperate Image & Caption
print(f'{captions[131]}')
img,caption = captions[131].split(",")
print(img)

pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)

In [ ]:
# seperate Image & Caption
print(f'{captions[1310]}')
img,caption = captions[1310].split(",")
print(img)

pathx = str(images_path+img)
# print(pathx)
img = cv2.imread(pathx)
img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(caption)

In [ ]:
import os # Handeling the file
import pickle #primarily used in serializing and deserializing a Python object structure
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm #used for creating Progress Meters or Progress Bars

from tensorflow.keras.preprocessing.image import load_img, img_to_array 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences # Even out the whole text representaion of features for the remaining length.
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout,add

In [ ]:
from nltk.translate.bleu_score import corpus_bleu # Bleu score

import cv2

import pdb #python debugger

import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Set directories
BASE_DIR='../input/flickr30k'
DATA_DIR='../input/flickr30k/Images'


## Libs

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences # Even out the whole text representaion of features for the remaining length.
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical,plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout,add

from nltk.translate.bleu_score import corpus_bleu # Bleu score


# Load Image Data

In [ ]:
import glob
path=glob.glob('../input/flickr30k/Images/*.jpg')
print(len(path))

## Model 1 - RESNET50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
resnet_model=ResNet50(weights='imagenet')
#re-structure the model by removing last 2 layers
resnet_model.layers.pop()
resnet_model=Model(inputs=resnet_model.inputs, outputs=resnet_model.layers[-2].output) ##Dropping last 2 layers
     

In [ ]:
print(resnet_model.summary())

### EXTRACT FEATURES WITH RESNET50

In [ ]:
features={}
#count=0
for img_name in tqdm(path):
 #load the images from Images folder
  img_file=cv2.imread(img_name)
  #resize data for model 
  img_file=cv2.resize(img_file,(224,224))
  #convert the image pixel to a numpy array
  X=image.img_to_array(img_file)
  #Reshape data for preprocess
  X=X.reshape((1,X.shape[0],X.shape[1],X.shape[2]))
    
  X=preprocess_input(X)
  #get features
  feature=resnet_model.predict(X,verbose=0).reshape(2048,)
  #Get image ID
  image_id=((os.path.basename(img_name)).split('.')[0])
  
  #Store features
  features[image_id]=feature

In [ ]:
WORKING_DIR = '/kaggle/working'
#Store featues in pickle
pickle.dump(features,open(os.path.join(WORKING_DIR,'features_resnet50.pk'),'wb'))

### Caption

In [ ]:
def Cleanup(mapping):
  import re
  for key, captions in mapping.items():
    for i in range(len(captions)):
      #take one cations at a time
      caption=captions[i]
      #convert to lowercase
      caption=caption.lower()
      #deleate digits,special chars
      caption=caption.replace('[^A-Za-z]', '')
      #delete addinional spaces
      caption=caption.replace('\s+', ' ')
      #Remove punctuations
      caption = re.sub(r'[^\w\s]', '', caption)
      #add start and end tags to the cation
      caption='startseq '+ ' '.join([word for word in caption.split() if len(word)>1]) +' endseq'
      captions[i]=caption
  return 

In [ ]:

with open(os.path.join(BASE_DIR,'captions.txt'),'rb') as f:
  next(f) # To remove 1st line "Image, captions"
  Cap_doc=f.read().decode('utf-8').split('\n')
Cap_doc[:10]
with open(os.path.join(BASE_DIR,'captions.txt'),'rb') as f:
  next(f) # To remove 1st line "Image, captions"
  Cap_doc=f.read().decode('utf-8').split('\n')
Cap_doc[:10]

In [ ]:
mapping={}

for line in tqdm(Cap_doc):
  #pdb.set_trace()
  #split line by whit space
  tokens=line.split(',')
  if len(line)<1:
    continue
  # Take the 1st token as the image id and the rest as the description
  image_id,image_desc=tokens[0],tokens[1:]
  # Remove filename from image id
  image_id=(image_id.split('.')[0])
  #onvert description tokens back to string
  image_desc=''.join(image_desc)
  #print(image_desc)
  #create list if needed
  if image_id not in mapping:
    mapping[image_id]=list()
    #mapping[image_id]=[]
  #store description
  mapping[image_id].append(image_desc) 

In [ ]:
Cleanup(mapping)

In [ ]:
mapping['1000268201']

In [ ]:
pickle.dump(mapping,open(os.path.join(WORKING_DIR,'Mapping.txt'),'wb'))

# LOAD IMAGE FEATURE & CAPTION

In [ ]:
with open(os.path.join(WORKING_DIR,'features_resnet50.pk'),'rb') as f:
  Img_features=pickle.load(f)
len(Img_features)
     

In [ ]:
with open(os.path.join(WORKING_DIR,'Mapping.txt'),'rb') as f:
  Mapping=pickle.load(f)
len(Mapping)
     

In [ ]:

all_captions=[]
for key in Mapping:
  for caption in Mapping[key]:
    all_captions.append(caption)
all_captions[:10]

In [ ]:
max_length_cap=max(len(caption.split())for caption in all_captions)
max_length_cap
     

In [ ]:

# tokenize the text 
tokenizer=Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_dict=tokenizer.word_index
vocab_size=len(vocab_dict)+1 # total number of unique word
vocab_size

In [ ]:
#Train test split
image_ids=list(Mapping.keys())
split=int(len(image_ids)*0.9)
train_id=image_ids[:split]
print(len(train_id))
#Train_id chunk 
train_id_1=train_id[:2500]
train_id_2=train_id[2500:5000]
train_id_3=train_id[5000:]

test_id=image_ids[split:]
print(len(train_id_1),len(train_id_2),len(train_id_3))

## DATAGEN

In [ ]:
def data_generator(data_keys,features,mapping,tokenizer,max_length,vocab_size,batch_size):
  X1,X2,y=[],[],[]
  n=0
  while 1:
    for key in data_keys:
      #print(key)
      n+=1
      captions=mapping[key]
      #print(captions)
      #process each captions
      for caption in captions:
        #encode the sequences
        seq=tokenizer.texts_to_sequences([caption])[0]
        #print(seq)
        #spli sequence into X,y pairs
        for i in range(1,len(seq)):
          #print(seq[i])
          #Split into input and output pairs
          in_seq,out_seq=seq[:i],seq[i]
          #print(in_seq,out_seq)
          #pad input seq
          in_seq=pad_sequences([in_seq],maxlen=max_length)[0]
          #encode output seq
          out_seq=to_categorical([out_seq],num_classes=vocab_size)[0]
          #print(in_seq)
          #print(out_seq)
          #store the seq
          X1.append(features[key])
          X2.append(in_seq)
          y.append(out_seq)
          #print(len(X1),len(X2),len(y))
          #print('X1:',X1)
          #print('X2:',X2)
          #print('y:',y)
      if n==batch_size:
        X1=np.array(X1)
        X2=np.array(X2)
        y=np.array(y)
        #print(X1.shape,X2.shape,y.shape)
        yield [X1,X2],y
        X1,X2,y=[],[],[]
        n=0
     

In [ ]:

#Encoder model
#image Feature layers
inputs1=Input(shape=(4096,))
Features1=Dropout(0.4)(inputs1)
Features2=Dense(256,activation='relu')(Features1)
#Text features layers
inputs2=Input(shape=(max_length_cap,))
seq1=Embedding(vocab_size,256,mask_zero=True)(inputs2)
seq2=Dropout(0.4)(seq1)
seq3=LSTM(256)(seq2)

#Decoder Model
decoder1=add([Features2,seq3])
decoder2=Dense(256,activation='relu')(decoder1)
outputs=Dense(vocab_size,activation='softmax')(decoder2)

model=Model(inputs=[inputs1,inputs2],outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

#plot the model
plot_model(model,show_shapes=True)

## V1 of model

In [ ]:

## Train model for total 55 epoch
epochs=15
batch_size=64
steps=(len(train_id)//batch_size)
for i in tqdm(range(epochs)):
  print('Epoch: ',i)
  generator=data_generator(train_id,Img_features,Mapping,tokenizer,max_length_cap,vocab_size,batch_size)
  model.fit(generator,epochs=1,steps_per_epoch=steps,shuffle=True,verbose=1)

In [ ]:
new_model.save(WORKING_DIR+'/best_model3.h5')